# Data Transforming 

This document record data transforming methods and transforming layer example. 

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
import importlib
import sys 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import clone

sys.path.append("../")

from proj_mod import visualization, data_processing
importlib.reload(visualization);
importlib.reload(data_processing);

## Categorizing the type of data 

| data | type | cyclical | encoding options | category | fall back |
|------|------|----------|------------------|----------|-----------| 
| age | numeric | NO | 1, keep as is 2, target encoding | personal | knn mean |
| job | categorical | NO | 1, one hot 2, target encoding| personal | mean |
| marital | categorical | NO | 1, one hot 2, target encoding | personal | mean |
| education | ordinal categorical | NO | 1, ordinal 2, target encoding | personal | mean | 
| default | binary | NO | 1, one hot 2, target encoding | personal | mean | 
| balance | numeric | NO | 1, log 2, target encoding | personal | knn mean | 
| housing | binary | NO | 1, one hot 2, target encoding | personal | mean | 
| loan | binary | NO | 1, one hot 2, target encoding | personal | mean | 
| contact | categorical | NO | 1, one hot 2, target encoding | campaign | mean | 
| day | numeric | YES | 1, keep as is 2, target encoding | campaign | knn mean | 
| month | categorical | YES | 1, one hot 2, target encoding | campaign | knn mean | 
| day of year (engineered) | numeric | YES | 1, keep as is 2, target encoding | campaign | knn mean | 
| duration | numeric | NO | 1, log 2, target encoding | campaign | knn mean | 
| campaign | numeric | NO | 1, keep as is 2, target encoding | campaign | knn mean | 

* Remark: There will be an option to "clean binary" since we only need to keep one of "{binary feature}_1", "{binary feature}_0", and "{binary feature}_target". 

We will first categorize by "category" by "personal" and "campaign". 
Under each, we will provide options to perform the default encoding (marked as 1 in "encoding options"), (inclusive) or target encoding. 

In [21]:
df=pd.read_csv("../data/raw.csv")
df["y"]=df["y"].map({"no": 0, "yes": 1})
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,technician,married,tertiary,no,395,no,no,cellular,3,jun,107,1,0
39996,30,management,single,tertiary,no,3340,no,no,cellular,3,jun,238,3,1
39997,54,admin,divorced,secondary,no,200,no,no,cellular,3,jun,170,1,1
39998,34,management,married,tertiary,no,1047,no,no,cellular,3,jun,342,1,0


In [22]:
features=df.columns[:-1]
df_feat=df[features]
df_tar=df[["y"]]

In [4]:
df_feat

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,technician,married,tertiary,no,395,no,no,cellular,3,jun,107,1
39996,30,management,single,tertiary,no,3340,no,no,cellular,3,jun,238,3
39997,54,admin,divorced,secondary,no,200,no,no,cellular,3,jun,170,1
39998,34,management,married,tertiary,no,1047,no,no,cellular,3,jun,342,1


In [5]:
df_tar

,y
0,0
1,0
2,0
3,0
4,0
...,...
39995,0
39996,1
39997,1
39998,0


In [8]:
data_transform_layer=data_processing.data_transform()
data_transform_layer.fit(X=df_feat,y=df_tar)

In [9]:
df_feat_transformed=data_transform_layer.transform(X=df_feat)
df_feat_transformed

,age,balance,education,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,...,month_num_8,month_num_10,month_num_11,month_num_12,contact_tar,day_tar,month_num_tar,day_of_year_tar,duration_tar,campaign_tar
0,58,7.670475,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.893459,7.319346,9.248389,13.196373,3.994450,6.853275
1,44,3.404525,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.893459,7.319346,9.248389,13.196373,2.667762,6.853275
2,33,1.131402,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.893459,7.319346,9.248389,13.196373,0.718313,6.853275
3,47,7.317943,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.893459,7.319346,9.248389,13.196373,1.120063,6.853275
4,33,0.741937,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.893459,7.319346,9.248389,13.196373,2.729112,6.853275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,5.981667,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.962832,10.906423,6.017752,7.475876,1.559663,6.853275
39996,30,8.114055,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.962832,10.906423,6.017752,7.475876,3.827409,6.853275
39997,54,5.303802,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.962832,10.906423,6.017752,7.475876,2.723738,6.853275
39998,34,6.954734,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.962832,10.906423,6.017752,7.475876,5.335033,6.853275


In [10]:
df_transformed=pd.concat([df_feat_transformed, df_tar], axis=1)
df_transformed

,age,balance,education,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,...,month_num_10,month_num_11,month_num_12,contact_tar,day_tar,month_num_tar,day_of_year_tar,duration_tar,campaign_tar,y
0,58,7.670475,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.893459,7.319346,9.248389,13.196373,3.994450,6.853275,0
1,44,3.404525,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.893459,7.319346,9.248389,13.196373,2.667762,6.853275,0
2,33,1.131402,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.893459,7.319346,9.248389,13.196373,0.718313,6.853275,0
3,47,7.317943,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.893459,7.319346,9.248389,13.196373,1.120063,6.853275,0
4,33,0.741937,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.893459,7.319346,9.248389,13.196373,2.729112,6.853275,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,5.981667,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.962832,10.906423,6.017752,7.475876,1.559663,6.853275,0
39996,30,8.114055,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.962832,10.906423,6.017752,7.475876,3.827409,6.853275,1
39997,54,5.303802,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.962832,10.906423,6.017752,7.475876,2.723738,6.853275,1
39998,34,6.954734,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.962832,10.906423,6.017752,7.475876,5.335033,6.853275,0


In [86]:
X_tr, X_te, y_tr, y_te = train_test_split(df_feat, df_tar, test_size=0.2)

In [87]:
X_te.shape

(8000, 13)

In [88]:
transform=data_processing.data_transform()
transform.fit(X=X_tr, y=y_tr)
out=transform.transform(X=X_te)
out

,age,balance,education,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,default_yes,housing_yes,loan_yes,job_tar,marital_tar,education_tar,age_tar,balance_tar,day,day_of_year,campaign,duration,contact_telephone,contact_unknown,month_num_2,month_num_3,month_num_4,month_num_5,month_num_6,month_num_7,month_num_8,month_num_10,month_num_11,month_num_12,contact_tar,day_tar,month_num_tar,day_of_year_tar,duration_tar,campaign_tar
0,51,4.522875,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.730952,6.165792,6.748393,5.870283,6.437315,19,231,2,5.416100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.947871,6.261024,17.170521,6.168160,3.471213,6.948063
1,54,6.131444,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.730952,8.497246,6.748393,5.619391,6.321292,20,324,3,4.890349,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.321041,7.166582,23.359610,11.168051,1.919049,6.948063
2,36,6.478663,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,5.783133,6.165792,5.737052,6.938611,7.476322,12,132,2,5.141664,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.916983,8.442702,9.136431,5.294551,2.722193,6.948063
3,44,5.774862,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,5.783133,6.165792,5.737052,6.430841,6.984379,11,162,8,4.094345,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.916983,8.026117,6.002971,5.613652,0.282768,5.631013
4,56,7.771109,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.413081,8.497246,9.190519,5.985618,11.392946,23,204,1,5.932245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.947871,8.952052,5.985414,6.508622,7.581882,6.948063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,37,3.763523,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,8.967083,9.083629,6.748393,6.736713,5.093611,22,234,2,4.605170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.947871,7.936438,17.170521,6.474005,1.519633,6.948063
7996,41,6.192567,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.730952,6.165792,6.748393,6.419960,7.264992,11,192,2,5.187386,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.321041,8.026117,5.985414,6.498950,2.692763,6.948063
7997,36,8.528746,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,8.118812,6.165792,9.190519,6.938611,13.105846,17,321,1,4.262680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.947871,6.761266,23.359610,14.767356,0.604473,6.948063
7998,31,8.354227,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.118812,9.083629,9.190519,8.291933,9.933687,18,230,3,4.882802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.947871,6.234254,17.170521,6.230313,1.892309,6.948063


In [23]:
SKF=StratifiedKFold(n_splits=5, shuffle=True, random_state=420)
splits=list(SKF.split(X=df_feat, y=df_tar))
df_list=[]
for tr_id, te_id in splits: 
    X_tr, X_te=df_feat.iloc[tr_id], df_feat.iloc[te_id]
    y_tr, y_te=df_tar.iloc[tr_id], df_tar.iloc[te_id]
    
    pipe=Pipeline([
        ("trans", data_processing.data_transform())
    ])
    
    in_splits=list(SKF.split(X=X_tr, y=y_tr))
    for in_tr, in_te in in_splits: 
        in_pipe=clone(pipe)
        X_tr_tr,X_tr_te=X_tr.iloc[in_tr], X_tr.iloc[in_te]
        y_tr_tr,y_tr_te=np.ravel(y_tr.iloc[in_tr].values), np.ravel(y_tr.iloc[in_te].values)
    
        in_pipe.fit(X=X_tr,y=y_tr)
        
        out=in_pipe.transform(X=X_te)
        
        df_list.append(out)

In [24]:
for index in range(len(df_list)): 
    print(index,":",df_list[index].columns[df_list[index].isnull().any()])

0 : Index([], dtype='object')
1 : Index([], dtype='object')
2 : Index([], dtype='object')
3 : Index([], dtype='object')
4 : Index([], dtype='object')
5 : Index([], dtype='object')
6 : Index([], dtype='object')
7 : Index([], dtype='object')
8 : Index([], dtype='object')
9 : Index([], dtype='object')
10 : Index([], dtype='object')
11 : Index([], dtype='object')
12 : Index([], dtype='object')
13 : Index([], dtype='object')
14 : Index([], dtype='object')
15 : Index([], dtype='object')
16 : Index([], dtype='object')
17 : Index([], dtype='object')
18 : Index([], dtype='object')
19 : Index([], dtype='object')
20 : Index([], dtype='object')
21 : Index([], dtype='object')
22 : Index([], dtype='object')
23 : Index([], dtype='object')
24 : Index([], dtype='object')
